In [ ]:
###############################################################################
import aliases
from everest.utilities import textutils
# tuple(textutils.find_all('', path='../everest/ptolemaic'))
# textutils.replace_all('Schematic', 'Compound', path='../everest')

In [ ]:
from everest.ptolemaic import chora
from everest.ptolemaic.sprite import Sprite

In [ ]:
class MyChora(chora.Choret):

    lower: POS[int]
    upper: POS[int] = None

    @classmethod
    def __parameterise__(cls, /, *args, **kwargs):
        params = super().__parameterise__(*args, **kwargs)
        if params.upper is None:
            params.upper = params.lower
            params.lower = 0
        return params

    def _incise_handle_int_(self, incisor: int, /):
        return chora.IncisionStyle.RETRIEVE(incisor)

    def _incise_handle_slice_(self, incisor: slice, /):
        start, stop, step = incisor.start, incisor.stop, incisor.step
        if step is not None:
            raise ValueError(incisor)
        return chora.IncisionStyle.SLYCE((start, stop))

    def _incise_handle_ellipsis_(self, incisor: type(Ellipsis), /):
        return chora.IncisionStyle.TRIVIAL(incisor)

    def _incise_retrieve_int_(self, incisor: int, /):
        return self.lower + incisor if incisor >= 0 else self.upper + incisor

    def _incise_slyce_tuple_(self, lower, upper, /):
        slower, supper = self.params
        return self.__ptolemaic_class__(
            slower if lower is None else \
                (slower if lower >= 0 else supper) + lower,
            supper if upper is None else \
                (slower if upper >= 0 else supper) + upper,
            )

    @organ
    def _incise_trivial_(self, /):
        def func(incisor: ..., /):
            return self
        return func

    def __iter__(self, /):
        return iter(range(*self.params))

In [ ]:
achora = MyChora(3, 9)
bchora = MyChora(5, 15)
cchora = bchora[achora]

In [ ]:
cchora[2:][3]